# Решение тестового задания на позицию "Junior Data Scientist"

# Решение 1 (SQL)
1. Вывести актуальное состояние товаров на 2020-06-01
Нужно выбрать самые свежие записи для каждого item_id, которые были обновлены до 2020-06-01.

1. Подготовка данных

In [1]:
import sqlite3
import pandas as pd

# Создаём соединение с базой данных в памяти (для тестирования)
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Создаём таблицу items
cursor.execute('''CREATE TABLE items (
    item_id INTEGER,
    name TEXT,
    price INTEGER,
    update_date DATE
)''')

# Заполняем таблицу items
cursor.executemany('''INSERT INTO items (item_id, name, price, update_date) VALUES (?, ?, ?, ?)''', [
    (1, "Ручка гелевая", 10, "2020-02-01"),
    (2, "Карандаш 1HH", 2, "2020-01-01"),
    (1, "Ручка шариковая", 10, "2020-03-01"),
    (3, "Ластик", 5, "2020-07-01"),
    (2, "Карандаш 1HH", 3, "2020-05-01"),
    (1, "Ручка шариковая", 5, "2020-05-01"),
    (2, "Карандаш 1H", 7, "2020-06-01")
])

# Создаём таблицу orders
cursor.execute('''CREATE TABLE orders (
    order_id INTEGER,
    user_id INTEGER,
    item_id INTEGER,
    order_date DATE
)''')

# Заполняем таблицу orders
cursor.executemany('''INSERT INTO orders (order_id, user_id, item_id, order_date) VALUES (?, ?, ?, ?)''', [
    (1, 1, 1, "2020-02-01"),
    (2, 2, 2, "2020-02-01"),
    (3, 1, 3, "2020-07-01"),
    (4, 3, 2, "2020-07-01"),
    (5, 2, 1, "2020-04-01"),
    (6, 1, 1, "2020-06-01")
])

print("Данные успешно загружены!")


Данные успешно загружены!


1.1. Актуальное состояние товаров на 2020-06-01

In [2]:
query_1 = '''
WITH latest_items AS (
    SELECT
        item_id,
        name,
        price,
        update_date,
        ROW_NUMBER() OVER (PARTITION BY item_id ORDER BY update_date DESC) AS rn
    FROM items
    WHERE update_date <= '2020-06-01'
)
SELECT item_id, name, price, update_date
FROM latest_items
WHERE rn = 1;
'''

df_1 = pd.read_sql_query(query_1, conn)
print("1. Актуальное состояние товаров на 2020-06-01:")
print(df_1)


1. Актуальное состояние товаров на 2020-06-01:
   item_id             name  price update_date
0        1  Ручка шариковая      5  2020-05-01
1        2      Карандаш 1H      7  2020-06-01


2. Товары, купленные по цене ≥ 3

In [3]:
query_2 = '''
WITH latest_prices AS (
    SELECT
        item_id,
        price,
        ROW_NUMBER() OVER (PARTITION BY item_id ORDER BY update_date DESC) AS rn
    FROM items
)
SELECT DISTINCT o.item_id
FROM orders o
JOIN latest_prices lp ON o.item_id = lp.item_id
WHERE lp.price >= 3 AND lp.rn = 1;
'''

df_2 = pd.read_sql_query(query_2, conn)
print("\n2. Товары, купленные по цене ≥ 3:")
print(df_2)



2. Товары, купленные по цене ≥ 3:
   item_id
0        1
1        2
2        3


3. Сумма покупок клиента 1

In [4]:
query_3 = '''
WITH latest_prices AS (
    SELECT
        item_id,
        price,
        ROW_NUMBER() OVER (PARTITION BY item_id ORDER BY update_date DESC) AS rn
    FROM items
)
SELECT SUM(lp.price) AS total_spent
FROM orders o
JOIN latest_prices lp ON o.item_id = lp.item_id
WHERE o.user_id = 1 AND lp.rn = 1;
'''

df_3 = pd.read_sql_query(query_3, conn)
print("\n3. Сумма покупок клиента 1:")
print(df_3)



3. Сумма покупок клиента 1:
   total_spent
0           15


4. Сумма всех покупок до 2020-05-01 включительно

In [5]:
query_4 = '''
WITH latest_prices AS (
    SELECT
        item_id,
        price,
        ROW_NUMBER() OVER (PARTITION BY item_id ORDER BY update_date DESC) AS rn
    FROM items
)
SELECT SUM(lp.price) AS total_spent
FROM orders o
JOIN latest_prices lp ON o.item_id = lp.item_id
WHERE o.order_date <= '2020-05-01' AND lp.rn = 1;
'''

df_4 = pd.read_sql_query(query_4, conn)
print("\n4. Сумма всех покупок до 2020-05-01:")
print(df_4)



4. Сумма всех покупок до 2020-05-01:
   total_spent
0           17


5. Сумма всех заказов и средняя цена заказа поквартально



In [6]:
query_5 = '''
WITH latest_prices AS (
    SELECT
        item_id,
        price,
        ROW_NUMBER() OVER (PARTITION BY item_id ORDER BY update_date DESC) AS rn
    FROM items
)
SELECT
    strftime('%Y-Q%m', o.order_date) AS quarter,
    COUNT(*) AS total_orders,
    SUM(lp.price) AS total_revenue,
    AVG(lp.price) AS avg_price
FROM orders o
JOIN latest_prices lp ON o.item_id = lp.item_id
WHERE lp.rn = 1
GROUP BY quarter;
'''

df_5 = pd.read_sql_query(query_5, conn)
print("\n5. Сумма всех заказов и средняя цена заказа поквартально:")
print(df_5)



5. Сумма всех заказов и средняя цена заказа поквартально:
    quarter  total_orders  total_revenue  avg_price
0  2020-Q02             2             12        6.0
1  2020-Q04             1              5        5.0
2  2020-Q06             1              5        5.0
3  2020-Q07             2             12        6.0


6. Оптимизация запросов для больших данных
* Индексы: Добавить индексы на item_id, update_date, order_date для быстрого поиска.
* Партиционирование: Разделить таблицы по order_date (например, по кварталам).
* Materialized Views: Создать представления с предрассчитанными актуальными ценами.
* Columnar Storage: Использовать ClickHouse или BigQuery для аналитики.


# Закрытие соединения с базой

In [7]:
conn.close()
print("Соединение с базой закрыто.")

Соединение с базой закрыто.


Решение №2 (Архитектура DWH и Data Vault 2.0Задание 2 (Архитектура DWH и Data Vault 2.0)

1. Построение Data Vault 2.0
* Hub (Сущности): hub_orders (уникальные ID заказов), hub_users (клиенты), hub_items (товары).
* Satellite (История изменений): sat_orders (изменения статусов, состав заказа), sat_prices (изменения цен).
* Link (Связи): link_order_items (какие товары в каких заказах).

2. Выделяем сущности                                                 

1.hub_orders (заказы)

2.hub_users (пользователи)

3.hub_items (товары)

4.sat_orders (изменения заказов)

5.sat_prices (история изменения цен)

6.link_order_items (связи товаров и заказов)


3. Как обновлять без updated_at?
* Скользящее окно: Обновлять данные, которые изменялись за последние N дней.
* Хэш-ключи: Если хэш от новых данных отличается от предыдущего, записываем изменение.
* Log-based CDC (Change Data Capture): Анализировать поток заказов и фиксировать изменения.
